## 最終開発課題

### 機能要件
- 2値の分類(Classification)タスクを扱える
- カテゴリカル変数を指定するとone-hotエンコードを実行する
- モデル用データマートに施したのと同一データ前処理をスコア用データマートに対しても適用される
- モデル選択の評価指標を選択できる
- 複数アルゴリズムから指定の評価指標に従いベストモデルを選択できる
- 学習済みモデルを保存できる
- アルゴリズムランキングと性能評価指標が出力される
- 学習済みモデル(保存したモデル)を呼び出しスコア用データに対し予測確率を付与できる

### 開発課題認定プロセス(最終日にその場で実行・提出)
- 訓練用データと(正解データの無い)検証用データを配布します
- データ形式は第1カラムがID, 第2カラムがクラス変数, 第3カラム以降が特徴ベクトルの構成です
  - 検証用データの第2カラムは全て空白
- クラス変数は0/1のバイナリ値で予測確率を知りたいクラスは'1'とする
- ID(第1カラム)と予測確率(第2カラム)の2カラム構成の結果ファイルをCSV形式で提出(ヘッダーあり)してもらいます

## 必要と想定されるモジュールの読み込み

In [1]:
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

## クラス定義〜前処理〜
- Preprocessingクラス

In [2]:
class Preprocessing(object):
    
    def __init__(self):
        pass
    
# モデルデータとスコアデータを読み込み、データの形状、データの型、カラム構成をチェックする
    def data_check(self, x1, x2):
        print('model shape: {}\nscore shape: {}'.format(x1.shape, x2.shape))
        print('-' * 30)
        print('model dtypes:\n{}\n{}\nscore dtype:\n{}'.format(x1.dtypes, '-'*30, x2.dtypes))
        print('-' * 30)
        diff1 = set(x1.columns.values) - set(x2.columns.values)
        diff2 = set(x2.columns.values) - set(x1.columns.values)
        print('model data only: {}'.format(diff1))
        print('score data only:  {}'.format(diff2))
        if len(diff1) + len(diff2) == 0:
            print('All match')
    
# 訓練データを、特徴量と目的変数に分けた状態で出力する
    def train_split(self, x):
        data = x
        X = data.iloc[:, 2:]
        y = data.iloc[:, 1]
        
        return X, y
    
# 目的変数がカテゴリカルデータだった場合に、0/1表現に変換する 
    def binary(self, positive, negative, y):
        class_mapping = {positive:1, negative:0}
        y = y.map(class_mapping)
        
        return y

# one-hot encodingを行う、引数として処理を行いたいカラム名をリストで渡す
    def ohe(self, x, columns):
        y = pd.get_dummies(x, dummy_na=True, columns=columns)
        
        return y
        
# 欠損値補完を行う、strategyのデフォルトは平均値
    def impute(self, x, strategy='mean'): 
        imp = SimpleImputer(strategy=strategy)
        imp.fit(x)
        x_columns = x.columns.values
        y = pd.DataFrame(data=imp.transform(x),columns=x_columns)

        return y
    
# 特徴量選択を行う、特徴量選択の数は、デフォルトで10が設定されている
    def feature_select(self, x, y, n=10):
        selector = RFE(estimator=RandomForestClassifier(n_estimators=100,random_state=0),
                                   n_features_to_select=n,step=.05)
        selector.fit(x, y)
        x_columns = x.columns.values
        x_selected = selector.transform(x)
        x_selected = pd.DataFrame(data=selector.transform(x), columns=x_columns[selector.support_])
        
        return x_selected
    
# testデータに対して、idとyをdropする
    def test_split(self, x):
        data = x
        y = data.iloc[:, 2:]
        
        return y

## クラス定義〜評価〜
- Metricsクラス

In [3]:
class Metrics(object):
    
    def __init__(self):
        pass
    
    # アルゴリズムのランキングと性能評価指標を出力する
    def algo_rank(self, scores):
        test_scores = {}
        for test_key, test_value in scores.items():
            if test_key[1] == 'test':
                test_scores[test_key] = test_value
        
        print('テストデータに対する各データの性能評価指標\n{}'.format(pd.Series(test_scores).unstack()))
        print('='*30)
        print('ベストアルゴリズム：{}'.format(max(test_scores.items(), key = lambda x:x[1])[0][0]))

        return pd.Series(test_scores).unstack(), max(test_scores.items(), key = lambda x:x[1])[0][0]

In [4]:
# クラスのインスタンス化
pre = Preprocessing()
met = Metrics()

## 作業に取り掛かる前に、データの構造を把握する
- データの形状
- データの型
- カテゴリカル変数がどの程度あるのか
- 学習用データとテスト用データで、カラム名が異なるかどうか

In [5]:
# 訓練用データをtrain_data,検証用データをtest_dataとして読み込む
train_data = pd.read_csv('final_hr_analysis_train.csv')
test_data = pd.read_csv('final_hr_analysis_test.csv')

In [ ]:
# カテゴリ変数をリストで設定
# ohe_columns = ['Dependents',
               'Gender',
               'Married',
               'Education',
               'Self_Employed',
               'Property_Area']

# カテゴリ変数をobject型で読み込むための準備
# my_dtype = {'Dependents':object,
            'Gender':object,
            'Married':object,
            'Education':object,
            'Self_Employed':object,
            'Property_Area':object}

In [6]:
# 正しく読み込めたか確認
train_data.head()

,index,left,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,sales,salary
0,10438,0,0.53,0.52,2,135,4,0,0,technical,medium
1,9236,0,0.77,0.53,5,256,3,0,0,accounting,medium
2,818,1,0.89,0.79,3,149,2,0,0,support,medium
3,11503,0,0.64,0.63,3,156,6,1,0,support,low
4,11721,0,0.98,0.74,4,151,3,0,0,sales,medium


In [7]:
# 正しく読み込めたか確認
test_data.head()

,index,left,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,sales,salary
0,1670,NaN,0.44,0.57,2,141,3,0,0,product_mng,medium
1,13378,NaN,0.55,0.96,3,194,3,0,0,product_mng,medium
2,10233,NaN,0.72,0.67,5,210,2,0,0,management,medium
3,4719,NaN,0.96,0.75,4,177,2,0,0,IT,low
4,7003,NaN,0.96,0.54,3,198,3,0,0,support,low


In [8]:
# 配布されたデータの形状や、データの型をチェックする
pre.data_check(train_data, test_data)

model shape: (10499, 11)
score shape: (4500, 11)
------------------------------
model dtypes:
index                      int64
left                       int64
satisfaction_level       float64
last_evaluation          float64
number_project             int64
average_montly_hours       int64
time_spend_company         int64
Work_accident              int64
promotion_last_5years      int64
sales                     object
salary                    object
dtype: object
------------------------------
score dtype:
index                      int64
left                     float64
satisfaction_level       float64
last_evaluation          float64
number_project             int64
average_montly_hours       int64
time_spend_company         int64
Work_accident              int64
promotion_last_5years      int64
sales                     object
salary                    object
dtype: object
------------------------------
model data only: set()
score data only:  set()
All match


In [14]:
test_data.isnull().sum()

index                       0
left                     4500
satisfaction_level          0
last_evaluation             0
number_project              0
average_montly_hours        0
time_spend_company          0
Work_accident               0
promotion_last_5years       0
sales                       0
salary                      0
dtype: int64

## 訓練用データをX, yに切り分ける

In [9]:
# 必ず、カラムをドロップする前にこの処理を行う
X, y = pre.train_split(train_data)

In [10]:
# Xが正しく切り分けられたか確認
X.head(3)

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,sales,salary
0,0.53,0.52,2,135,4,0,0,technical,medium
1,0.77,0.53,5,256,3,0,0,accounting,medium
2,0.89,0.79,3,149,2,0,0,support,medium


In [11]:
# yが正しく切り分けられたか確認
y.head(3)

0    0
1    0
2    1
Name: left, dtype: int64

## 必要な変数を格納しておく。想定されるのは
- dropする不要なカラムリスト
- カテゴリカル変数のカラムリスト
- 欠損値補完をしたいカラムリスト

In [15]:
drop_columns = []
ohe_columns = ['sales','salary']

## 前処理フェーズ
- 目的変数を0/1表現にする
- 不要な特徴量をdropする
- カテゴリカルデータをone-hot encodingする
- 欠損値補完を行う
- 特徴量選択を行う

In [ ]:
# 目的変数を0/1表現にする
y = pre.binary('positive', 'negative')

In [ ]:
y

In [13]:
# 不要な特徴量をdropする
# drop_columns = []
X.drop(columns=drop_columns, inplace=True)

In [ ]:
X.head()

In [16]:
# カテゴリカルデータをone-hot encodingする
X_new = pre.ohe(X, ohe_columns)

In [17]:
X_new.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,sales_IT,sales_RandD,sales_accounting,...,sales_marketing,sales_product_mng,sales_sales,sales_support,sales_technical,sales_nan,salary_high,salary_low,salary_medium,salary_nan
0,0.53,0.52,2,135,4,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
1,0.77,0.53,5,256,3,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
2,0.89,0.79,3,149,2,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
3,0.64,0.63,3,156,6,1,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
4,0.98,0.74,4,151,3,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [19]:
X_new.shape

(10499, 22)

In [17]:
# 欠損値補完を行う
X_new = pre.impute(X_new, strategy='mean')

In [ ]:
X_new.head()

In [19]:
# 特徴量選択を行う
X_new = pre.feature_select(X_new, y, 5)
X_new.shape

(891, 5)

In [ ]:
X_new.head()

## 学習フェーズ
- モデル選択の評価指標を選択する
- 複数アルゴリズムを選択し、選択した評価指標に基づいてモデル評価を行う
- 学習したモデルを保存する
- アルゴリズムのランキングと性能評価指標を出力する

In [20]:
# 目的変数のバランスを見る
y.value_counts()

0    7966
1    2533
Name: left, dtype: int64

### 指定の評価指標を選択する

In [21]:
# モデル選択の評価指標を選択する(指示あり？)
# accuracy_score, precision_score, recall_score, f1_score
select_metrics = f1_score

## クロスバリデーションの指定があった場合

In [ ]:
# 複数アルゴリズムを選択し、選択した評価指標に基づいてモデル評価を行う
pipelines = {
    'knn':
        Pipeline([('scl',StandardScaler()),
                  ('est',KNeighborsClassifier())]),
    'logistic':
        Pipeline([('scl',StandardScaler()),
                  ('est',LogisticRegression(random_state=1))]),
    'rsvc':
        Pipeline([('scl',StandardScaler()),
                  ('est',SVC(C=1.0, kernel='rbf', class_weight='balanced', random_state=1))]),
    'tree':
        Pipeline([('scl',StandardScaler()),
                  ('est',DecisionTreeClassifier(random_state=1))]),
    'rf':
        Pipeline([('scl',StandardScaler()),
                  ('est',RandomForestClassifier(random_state=1))]),
    'gb':
        Pipeline([('scl',StandardScaler()),
                  ('est',GradientBoostingClassifier(random_state=1))]),
    'mlp':
        Pipeline([('scl',StandardScaler()),
                  ('est',MLPClassifier(hidden_layer_sizes=(3,3),
                                       max_iter=1000,
                                       random_state=1))])
}

scores = {}
kfold = KFold(n_splits=5, shuffle=True, random_state=0)
    
for pipe_name, pipeline in pipelines.items():
    score = cross_val_score(pipeline, X_new, y, cv=kfold, scoring='f1')
    scores[(pipe_name,'score')] = score.mean()

print('使用した評価指標：{}'.format(select_metrics))
pd.Series(scores).unstack()

## ホールドアウト法の指定があった場合(指定がない場合はこっちを使う)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X_new, y, random_state=0)

In [23]:
# 複数アルゴリズムを選択し、選択した評価指標に基づいてモデル評価を行う
pipelines = {
    'knn':
        Pipeline([('scl',StandardScaler()),
                  ('est',KNeighborsClassifier())]),
    'logistic':
        Pipeline([('scl',StandardScaler()),
                  ('est',LogisticRegression(random_state=1))]),
    'rsvc':
        Pipeline([('scl',StandardScaler()),
                  ('est',SVC(C=1.0, kernel='rbf', class_weight='balanced', random_state=1))]),
    'tree':
        Pipeline([('scl',StandardScaler()),
                  ('est',DecisionTreeClassifier(random_state=1))]),
    'rf':
        Pipeline([('scl',StandardScaler()),
                  ('est',RandomForestClassifier(random_state=1))]),
    'gb':
        Pipeline([('scl',StandardScaler()),
                  ('est',GradientBoostingClassifier(random_state=1))]),
    'mlp':
        Pipeline([('scl',StandardScaler()),
                  ('est',MLPClassifier(hidden_layer_sizes=(3,3),
                                       max_iter=1000,
                                       random_state=1))])
}

scores = {}
    
for pipe_name, pipeline in pipelines.items():
    pipeline.fit(X_train, y_train)
    scores[(pipe_name,'train')] = select_metrics(y_train, pipeline.predict(X_train))
    scores[(pipe_name,'test')] = select_metrics(y_test, pipeline.predict(X_test))

print('使用した評価指標：{}'.format(select_metrics))
pd.Series(scores).unstack()

使用した評価指標：<function f1_score at 0x1a1cfc8e60>


,test,train
gb,0.938617,0.953989
knn,0.875847,0.907173
logistic,0.471264,0.449680
mlp,0.887172,0.909774
rf,0.971206,0.999734
rsvc,0.864591,0.892612
tree,0.942771,1.000000


In [24]:
test_ranking, best_model = met.algo_rank(scores)

テストデータに対する各データの性能評価指標
              test
gb        0.938617
knn       0.875847
logistic  0.471264
mlp       0.887172
rf        0.971206
rsvc      0.864591
tree      0.942771
ベストアルゴリズム：rf


In [25]:
# 学習したモデルを保存する
pred_model = pipelines[best_model]

In [26]:
# モデルをpickleに保存
with open('model.pickle', mode='wb') as fp:
    pickle.dump(pred_model)

TypeError: dump() missing required argument 'file' (pos 2)

## 検証用データに学習用データと同様の処理を施す
- idとyをドロップする
- 不要な特徴量をdropする
- カテゴリカルデータをone-hot-encodingする
- 欠損値補完を行う
- X_newの特徴量と合わせる

In [27]:
# testdataからidとyをdropする
test_X = pre.test_split(test_data)

In [28]:
test_X.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,sales,salary
0,0.44,0.57,2,141,3,0,0,product_mng,medium
1,0.55,0.96,3,194,3,0,0,product_mng,medium
2,0.72,0.67,5,210,2,0,0,management,medium
3,0.96,0.75,4,177,2,0,0,IT,low
4,0.96,0.54,3,198,3,0,0,support,low


In [30]:
# 不要な特徴量をdropする
# drop_columns = []
test_X.drop(columns=drop_columns, inplace=True)

In [ ]:
test_X.head()

In [29]:
# カテゴリカルデータをone-hot encodingする
test_X_new = pre.ohe(test_X, ohe_columns)

In [30]:
test_X_new.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,sales_IT,sales_RandD,sales_accounting,...,sales_marketing,sales_product_mng,sales_sales,sales_support,sales_technical,sales_nan,salary_high,salary_low,salary_medium,salary_nan
0,0.44,0.57,2,141,3,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
1,0.55,0.96,3,194,3,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
2,0.72,0.67,5,210,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0.96,0.75,4,177,2,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0.96,0.54,3,198,3,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0


In [34]:
# 欠損値補完を行う
test_X_new = pre.impute(test_X_new, strategy='mean')

In [ ]:
test_X_new.head()

In [36]:
# X_newの特徴量と合わせる
test_X_new = test_X_new[X_new.columns]

In [ ]:
test_X_new.head()

## モデル用とスコア用のデータが一致しているか確認

In [31]:
# データの形状や、データの型をチェックする
pre.data_check(X_new, test_X_new)

model shape: (10499, 22)
score shape: (4500, 22)
------------------------------
model dtypes:
satisfaction_level       float64
last_evaluation          float64
number_project             int64
average_montly_hours       int64
time_spend_company         int64
Work_accident              int64
promotion_last_5years      int64
sales_IT                   uint8
sales_RandD                uint8
sales_accounting           uint8
sales_hr                   uint8
sales_management           uint8
sales_marketing            uint8
sales_product_mng          uint8
sales_sales                uint8
sales_support              uint8
sales_technical            uint8
sales_nan                  uint8
salary_high                uint8
salary_low                 uint8
salary_medium              uint8
salary_nan                 uint8
dtype: object
------------------------------
score dtype:
satisfaction_level       float64
last_evaluation          float64
number_project             int64
average_montly_hours   

## 予測フェーズ
- 保存したモデルで、検証用データの予測を確率で出力
- 検証用データに予測確率を付与する

In [39]:
# 保存したモデルで、検証用データの予測を確率で出力

In [41]:
pred_model.predict(test_X_new)

array([1, 0, 0, ..., 0, 0, 0])

In [32]:
y_pred = pred_model.predict_proba(test_X_new)[:, 1]
y_pred = pd.Series(data=y_pred, name='y_pred')

In [33]:
y_pred

0       0.99
1       0.00
2       0.01
3       0.00
4       0.00
        ... 
4495    0.02
4496    0.01
4497    0.06
4498    0.17
4499    0.02
Name: y_pred, Length: 4500, dtype: float64

In [44]:
# 検証用データに予測確率を付与する

In [34]:
submission_id = test_data.iloc[:, 0]

In [35]:
submission = pd.concat([submission_id, y_pred], axis=1)

In [36]:
submission.head()

,index,y_pred
0,1670,0.99
1,13378,0.00
2,10233,0.01
3,4719,0.00
4,7003,0.00


## 提出フェーズ
- クラス変数は0/1のバイナリ値で予測確率を知りたいクラスは'1'とする
- ID(第1カラム)と予測確率(第2カラム)の2カラム構成の結果ファイルをCSV形式で提出(ヘッダーあり)してもらいます

In [37]:
submission.to_csv('aijcf0602.csv', header=True)

In [40]:
a = pd.read_csv('aijcf0602.csv')
a

,Unnamed: 0,index,y_pred
0,0,1670,0.99
1,1,13378,0.00
2,2,10233,0.01
3,3,4719,0.00
4,4,7003,0.00
...,...,...,...
4495,4495,11275,0.02
4496,4496,3828,0.01
4497,4497,4645,0.06
4498,4498,6069,0.17
